In [1]:
import kagglehub
import torch
import os
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.utils.data import Dataset, DataLoader
from torch import nn,optim
from tqdm import tqdm
import wandb
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

In [2]:
# Download latest version
path = kagglehub.dataset_download("clmentbisaillon/fake-and-real-news-dataset")
print("Path to dataset files:", path)


Using Colab cache for faster access to the 'fake-and-real-news-dataset' dataset.
Path to dataset files: /kaggle/input/fake-and-real-news-dataset


In [3]:

os.listdir(path)

['True.csv', 'Fake.csv']

In [4]:
True_path=os.path.join(path, os.listdir(path)[0])
Fake_path=os.path.join(path, os.listdir(path)[1])
True_data=pd.read_csv(True_path)
Fake_data=pd.read_csv(Fake_path)

In [5]:
True_data.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [6]:
True_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21417 entries, 0 to 21416
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    21417 non-null  object
 1   text     21417 non-null  object
 2   subject  21417 non-null  object
 3   date     21417 non-null  object
dtypes: object(4)
memory usage: 669.4+ KB


In [7]:
Fake_data.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [8]:
Fake_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23481 entries, 0 to 23480
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    23481 non-null  object
 1   text     23481 non-null  object
 2   subject  23481 non-null  object
 3   date     23481 non-null  object
dtypes: object(4)
memory usage: 733.9+ KB


In [9]:
#Now, I want to join both the True_data table and the Fake_data table

# Firstly, I tag both the True_data and the Fake_data as 1 and 0 respectively
True_data['label']=1
Fake_data['label']=0

#Then, I concat the two tables
data=pd.concat([Fake_data, True_data]).reset_index(drop=True)

data.head()

,title,text,subject,date,label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0


In [10]:
data.tail()

,title,text,subject,date,label
44893,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,worldnews,"August 22, 2017",1
44894,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",worldnews,"August 22, 2017",1
44895,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,worldnews,"August 22, 2017",1
44896,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,worldnews,"August 22, 2017",1
44897,Indonesia to buy $1.14 billion worth of Russia...,JAKARTA (Reuters) - Indonesia will buy 11 Sukh...,worldnews,"August 22, 2017",1


In [11]:
#Now, let us combine the title column and the text into a new single column
data["content"]=data["title"] + " " + data["text"]

data.head()

,title,text,subject,date,label,content
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0,Donald Trump Sends Out Embarrassing New Year’...
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0,Drunk Bragging Trump Staffer Started Russian ...
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0,Sheriff David Clarke Becomes An Internet Joke...
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0,Trump Is So Obsessed He Even Has Obama’s Name...
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0,Pope Francis Just Called Out Donald Trump Dur...


In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44898 entries, 0 to 44897
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    44898 non-null  object
 1   text     44898 non-null  object
 2   subject  44898 non-null  object
 3   date     44898 non-null  object
 4   label    44898 non-null  int64 
 5   content  44898 non-null  object
dtypes: int64(1), object(5)
memory usage: 2.1+ MB


In [13]:
# download the tokenizer

model_uri = "distilbert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_uri)

# split the dataset

train_data, test_data = train_test_split(data, test_size=0.27, random_state=11, stratify=data['label'])

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/465 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [14]:
# set up the dataset proper

class News_Classifier(Dataset):

  def __init__(self, dataframe: pd.DataFrame, tokenizer, max_length=128) -> None:
    self.content = dataframe['content'].to_list()
    self.labels = dataframe['label'].to_list()
    self.max_length = max_length
    self.tokenizer = tokenizer

  def __len__(self):
    return len(self.labels)

  def __getitem__(self, index):
    content = self.content[index]
    label = self.labels[index]

    encoded_batch = self.tokenizer.encode_plus(
        text = content,
        add_special_tokens = True,
        padding = 'max_length',
        truncation = True,
        return_tensors = 'pt'
    )
    input_ids = encoded_batch['input_ids'].flatten()
    attention_mask = encoded_batch['attention_mask'].flatten()

    return {
        'input_ids': input_ids,
        'attention_mask': attention_mask,
        'label': torch.tensor(data=label, dtype=torch.long)
    }

In [15]:
train_data = News_Classifier(dataframe=train_data,tokenizer=tokenizer)
test_data = News_Classifier(dataframe=test_data,tokenizer=tokenizer)

In [16]:
# download the model, initialize it and get the hyperparams

model = AutoModelForSequenceClassification.from_pretrained(model_uri, num_labels=2)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = model.to(device)
criterion = nn.CrossEntropyLoss()
lr = 2e-5
optimizer = optim.AdamW(params=model.parameters(), lr=lr, weight_decay=0.01)
epochs = 5
device
batch_size = 16

model.safetensors:   0%|          | 0.00/263M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
# setup dataloader

train_loader = DataLoader(dataset=train_data, batch_size=batch_size)
test_loader = DataLoader(dataset=test_data, batch_size=batch_size)

In [18]:
# setup the training and validation loop
overall_train_loss = []
overall_test_loss = []
for epoch in range(epochs):
  model.train()
  epoch_labels = []
  epoch_preds = []
  epoch_test_labels = []
  epoch_test_preds = []
  epoch_train_loss = []
  epoch_test_loss = []
  train_batch = tqdm(train_loader, desc=f"Training Epoch {epoch + 1}/{epochs}...")
  for batch in train_batch:
    input_ids, attention_mask, labels = batch['input_ids'].to(device), batch['attention_mask'].to(device), batch['label'].to(device)
    train_preds = model(input_ids=input_ids,attention_mask=attention_mask, labels=labels)
    loss = train_preds.loss
    logit= train_preds.logits

    #backprop
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    epoch_train_loss.append(loss.item())
    epoch_labels.extend(labels.detach().cpu().numpy())
    logit, preds = torch.max(logit, 1)
    epoch_preds.extend(preds.detach().cpu().numpy())

  # compute metrics
  train_loss = sum(epoch_train_loss)/len(epoch_train_loss)
  train_f1 = f1_score(epoch_labels, epoch_preds, average = "weighted")
  print(f"Training Completed.....")

  # evaluation loop
  model.eval()
  with torch.no_grad():
    test_batch = tqdm(test_loader, desc=f"Testing epoch {epoch + 1}/ {epochs}")
    for batch in test_batch:
      input_ids,attention_mask, labels = batch["input_ids"].to(device),batch["attention_mask"].to(device), batch["label"].to(device)
      test_preds = model(input_ids=input_ids,attention_mask=attention_mask)
      logit_test = test_preds.logits

      # get all the needed result
      epoch_train_loss.append(loss.item())
      epoch_test_labels.extend(labels.detach().cpu().numpy())
      logit, preds = torch.max(logit_test, 1)
      epoch_test_preds.extend(preds.detach().cpu().numpy())

  #compute the test metrics
  test_f1 = f1_score(epoch_test_labels, epoch_test_preds, average = "weighted")

  epoch_result = {
      'epoch': epoch + 1,
      'train_loss': train_loss,
      'test_f1': test_f1,
      'train_f1': train_f1
  }
  print(f"Test Completed with result: {epoch_result}")

Training Epoch 1/5...: 100%|██████████| 2049/2049 [26:34<00:00,  1.29it/s]


Training Completed.....


Testing epoch 1/ 5: 100%|██████████| 758/758 [03:13<00:00,  3.92it/s]


Test Completed with result: {'epoch': 1, 'train_loss': 0.009998816142564263, 'test_f1': 1.0, 'train_f1': 0.9961551269869303}


Training Epoch 2/5...: 100%|██████████| 2049/2049 [26:28<00:00,  1.29it/s]


...

Test Completed with result: {'epoch': 4, 'train_loss': 4.426677890939198e-06, 'test_f1': 1.0, 'train_f1': 1.0}


Training Epoch 5/5...: 100%|██████████| 2049/2049 [26:27<00:00,  1.29it/s]


Training Completed.....


Testing epoch 5/ 5: 100%|██████████| 758/758 [03:12<00:00,  3.93it/s]

Test Completed with result: {'epoch': 5, 'train_loss': 1.289616588436023e-06, 'test_f1': 1.0, 'train_f1': 1.0}


In [19]:
from huggingface_hub import login
from IPython.display import clear_output

my_token=input("Enter the token: ")
clear_output()

login(token=my_token)

In [20]:
repo_id= "News_Classifier"
model.push_to_hub(repo_id)
tokenizer.push_to_hub(repo_id)

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  /tmp/tmpbglsusw7/model.safetensors    :   0%|          | 10.8kB /  263MB            

README.md: 0.00B [00:00, ?B/s]

CommitInfo(commit_url='https://huggingface.co/salamichino/News_Classifier/commit/a96a891dcfb57b1aad1ee32080907ef79496e2bb', commit_message='Upload tokenizer', commit_description='', oid='a96a891dcfb57b1aad1ee32080907ef79496e2bb', pr_url=None, repo_url=RepoUrl('https://huggingface.co/salamichino/News_Classifier', endpoint='https://huggingface.co', repo_type='model', repo_id='salamichino/News_Classifier'), pr_revision=None, pr_num=None)